In [ ]:
# Melanoma Detection Using Custom CNN
This notebook demonstrates the process of building, training, and evaluating a custom CNN for melanoma detection from images.

In [ ]:
# Import libraries
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import numpy as np
import os
from sklearn.utils import class_weight
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## Load and Preprocess Dataset
### Define Paths and Load Dataset

In [ ]:
# Define dataset paths
train_dir = '/CaseStudy/Skin cancer ISIC The International Skin Imaging Collaboration/Train'
val_dir = '/CaseStudy/Skin cancer ISIC The International Skin Imaging Collaboration/Test'

# Load datasets
train_ds = tf.keras.utils.image_dataset_from_directory(train_dir, image_size=(180, 180), batch_size=32)
val_ds = tf.keras.utils.image_dataset_from_directory(val_dir, image_size=(180, 180), batch_size=32)

In [ ]:
# Normalize pixel values to [0, 1]
normalization_layer = layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))

In [ ]:
## Visualize Dataset
### Display One Image Per Class

# Visualize one image from each class
class_names = train_ds.class_names
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    for i in range(9):  # Adjust based on the number of classes
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy())
        plt.title(class_names[labels[i]])
        plt.axis("off")
plt.show()

In [ ]:
## Build Baseline Model

In [ ]:
# Define a simple CNN model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(180, 180, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dropout(0.5),
    layers.Dense(128, activation='relu'),
    layers.Dense(9, activation='softmax')
])

In [ ]:
## Compile and Train the Baseline Model

In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(train_ds, validation_data=val_ds, epochs=20)

In [ ]:
## Evaluate Baseline Model
### Plot Training and Validation Curves

In [ ]:
# Plot training and validation curves
plt.figure(figsize=(10, 5))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training and Validation Accuracy')
plt.show()

plt.figure(figsize=(10, 5))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
## Apply Data Augmentation

In [ ]:
# Add data augmentation layers
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2)
])

# Apply data augmentation to training dataset
augmented_train_ds = train_ds.map(lambda x, y: (data_augmentation(x, training=True), y))

In [ ]:
## Re-train Model with Augmented Data

In [ ]:
# Re-compile and train the model
history_augmented = model.fit(augmented_train_ds, validation_data=val_ds, epochs=20)

In [ ]:
## Analyze Class Distribution and Handle Imbalances

In [ ]:
# Analyze class distribution
class_counts = {class_name: 0 for class_name in class_names}
for _, labels in train_ds.unbatch():
    for label in labels.numpy():
        class_counts[class_names[label]] += 1

# Plot class distribution
plt.figure(figsize=(10, 5))
plt.bar(class_counts.keys(), class_counts.values())
plt.title('Class Distribution')
plt.xlabel('Classes')
plt.ylabel('Number of Samples')
plt.xticks(rotation=45)
plt.show()

In [ ]:
## Re-train Model with Balanced Data

In [ ]:
# Use Augmentor to balance classes (pseudo-code for now)
# Augment images for underrepresented classes
 balanced_train_ds = augmentor_balance_classes(train_dir)

# Train the model on balanced data
 history_balanced = model.fit(balanced_train_ds, validation_data=val_ds, epochs=30)

In [ ]:
## Compare Results and Final Analysis

In [ ]:
# Compare results for baseline, augmented, and balanced models
# Example: Print accuracy for each model
 print("Baseline Accuracy:", history.history['val_accuracy'][-1])
 print("Augmented Accuracy:", history_augmented.history['val_accuracy'][-1])
 print("Balanced Accuracy:", history_balanced.history['val_accuracy'][-1])